# Søk om utsatt frist for et set av klienter

In [2]:
from docs.test.testinnsending.skatteetaten_api import get_access_token
from requests import post
from faker import Faker
import jwt
import uuid

token = get_access_token()
jwt_token = token["Authorization"].replace("Bearer ", "")


#henter innlogget bruker fra tokenet
identifikator = jwt.decode(jwt_token, options={"verify_signature": False})["sub"]
inntektsår = "2025"

AssertionError: 

In [3]:
def lag_indentifikator(identifikator: str) -> dict():
    fake = Faker()

    if len(identifikator) > 9:
        identifikator_dict = {"foedselsnummer" : identifikator}
    else:
        identifikator_dict = {"organisasjonsnummer" : identifikator}

    kandidat = {
        "identifikator": identifikator_dict,
        "naering":  True, #kan ikke mikse True / False av næring i en og samme request
        "navn": fake.name(),
        "epost": fake.email(),
    }
    return kandidat


parter = ["21909899726", "18853449683"]

klienter = [lag_indentifikator(kandidat) for kandidat in parter]

payload = {"referanse": str(uuid.uuid4()),
           "klienter": klienter}

url_path = f"/api/skattemelding/v2/utsattfristsoeknad/bulk/{identifikator}"
base_url = "https://idporten-api-test.sits.no"

In [3]:
utsatt_frist_request = post(base_url+url_path, headers=token, json=payload)
print(f"http: {utsatt_frist_request.status_code}")
print(utsatt_frist_request.text)

http: 500
{"errorMessage":"underliggende feil mot utsattfrist mottak","feilkode":"SME-EKSTERN-002","korrelasjonsId":"132151dc-ca0f-e7ed-23af-8293f3c94f72"}


In [4]:
url_path = f"/api/skattemelding/v2/utsattfristsoeknad/{identifikator}"

payload = {"referanse": str(uuid.uuid4()),
           "klient": lag_indentifikator("21909899726")}

utsatt_frist_request_singel = post(base_url+url_path, headers=token, json=payload)
print(f"http: {utsatt_frist_request_singel.status_code}")
print(utsatt_frist_request_singel.text)

http: 200
{"gjeldendeInntektsaar":2024,"innsender":"10894799493","klient":"21909899726","beskrivelse":"Ingen endring ble gjort for innsendt skattepliktig."}
